## Install Dependencies

In [1]:
!pip install langchain langchain-community langchain-core pinecone fastembed langchain-pinecone

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.7/84.7 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.4/476.4 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 45.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.8/324.

In [1]:
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone, ServerlessSpec
from sentence_transformers import SentenceTransformer
from langchain.embeddings.base import Embeddings
from google.colab import userdata
import time

## Load embedding model

In [2]:
class LaBSEEmbeddings(Embeddings):
    def __init__(self, model_name: str = 'sentence-transformers/LaBSE'):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts):
        return [self.model.encode(text).tolist() for text in texts]

    def embed_query(self, text):
        return self.model.encode(text).tolist()

embedding_model = LaBSEEmbeddings()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

2_Dense/model.safetensors:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

In [3]:
embedding_model = LaBSEEmbeddings()

## Create index in vector store

In [4]:
pc = Pinecone(api_key=userdata.get('PINECONE_API_KEY'))
existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]
pc_index_name = "gic-labse"

if pc_index_name in existing_indexes:
  print(f"Vector store index '{pc_index_name}' found. Deleting existing...")
  pc.delete_index(pc_index_name)


print(f"Creating new vector store...")

pc.create_index(
  name=pc_index_name,
  dimension=768,
  metric="cosine",
  spec=ServerlessSpec(
    cloud="aws",
    region="us-east-1"
  )
)

while not pc.describe_index(pc_index_name).status['ready']:  # Wait for the index to be ready
  time.sleep(1)
  print(f"Successfully created vector store index '{pc_index_name}'.")


Vector store index 'gic-labse' found. Deleting existing...
Creating new vector store...


## Store contexts in vector store index

In [5]:
import pandas as pd

data = pd.read_csv("gic_data_contexts_final_with_meta.csv")
print(len(data))

1699


In [6]:
data.head()

,context_id,context_title,context_content,main_category_id,main_category_name,sub_category_name,sub_category_id,service_name,service_id,service_link,context_with_metadata
0,1-87-665-1,Educational Publications Department Book Sales...,"There are 6 sales outlets of books, successful...",1,Education & Training,Education Publications,87,Sales Outlets of Books,665,https://gic.gov.lk/gic/index.php/en/component/...,Service: Sales Outlets of Books\nMain Category...
1,1-87-1481-1,Publication Prices - Sinhala Publications,Publication Price Sinhala Publications Rs. Cts...,1,Education & Training,Education Publications,87,Museum Publications,1481,https://gic.gov.lk/gic/index.php/en/component/...,Service: Museum Publications\nMain Category: E...
2,1-87-1481-2,Publication Prices - English Publications,English Publications\n21. Some Sinhala Combati...,1,Education & Training,Education Publications,87,Museum Publications,1481,https://gic.gov.lk/gic/index.php/en/component/...,Service: Museum Publications\nMain Category: E...
3,1-87-1481-3,Publication Prices - English and Tamil Publica...,36. The Pleistocene of Ceylon 390 00\n37. Colo...,1,Education & Training,Education Publications,87,Museum Publications,1481,https://gic.gov.lk/gic/index.php/en/component/...,Service: Museum Publications\nMain Category: E...
4,1-87-662-1,Departmental Administration and Record Keeping,1. Maintaining the personal files of the offic...,1,Education & Training,Education Publications,87,Services of Establishment Unit of Educational ...,662,https://gic.gov.lk/gic/index.php/en/component/...,Service: Services of Establishment Unit of Edu...


In [7]:
from langchain_core.documents import Document

docs = []

for i, row in data.iterrows():
    if i % 20 == 0:
        print(f"Adding context {i}")

    doc = Document(
        page_content=row["context_with_metadata"],
        metadata={
            "context_id": row["context_id"],
            "context_title": row["context_title"],
            "main_category_id": int(row["main_category_id"]),
            "main_category_name": row["main_category_name"],
            "sub_category_id": int(row["sub_category_id"]),
            "sub_category_name": row["sub_category_name"],
            "service_id": int(row["service_id"]),
            "service_name": row["service_name"],
            "service_link": row["service_link"],
            "language": "en"
        }
    )

    docs.append(doc)

print(len(docs))

Adding context 0
Adding context 20
Adding context 40
Adding context 60
Adding context 80
Adding context 100
Adding context 120
Adding context 140
Adding context 160
Adding context 180
Adding context 200
Adding context 220
Adding context 240
Adding context 260
Adding context 280
Adding context 300
Adding context 320
Adding context 340
Adding context 360
Adding context 380
Adding context 400
Adding context 420
Adding context 440
Adding context 460
Adding context 480
Adding context 500
Adding context 520
Adding context 540
Adding context 560
Adding context 580
Adding context 600
Adding context 620
Adding context 640
Adding context 660
Adding context 680
Adding context 700
Adding context 720
Adding context 740
Adding context 760
Adding context 780
Adding context 800
Adding context 820
Adding context 840
Adding context 860
Adding context 880
Adding context 900
Adding context 920
Adding context 940
Adding context 960
Adding context 980
Adding context 1000
Adding context 1020
Adding context 1

In [8]:
index = pc.Index(pc_index_name)
vector_store = PineconeVectorStore(index=index, embedding=embedding_model)

In [9]:
vector_store.add_documents(
    documents=docs,
    batch_size=50
)

['4ca8b201-d9d7-4cda-acbc-2b8803e2b4dc',
 'f2af53e1-9523-40a6-a94e-1d8b8ad906af',
 '7d33e86f-62df-4265-841d-ab248dfc8086',
 'fe845a38-ea9e-499d-a5c7-230c17015f60',
 '5c92727e-6265-4761-89a1-3366250c2be3',
 '47bdbc7c-ab15-4f91-8641-cd3ec8850d5a',
 '80c7bd5c-626e-4539-8ba1-890208004775',
 '5aa8655c-5137-483a-9471-6e6fa14ab6d8',
 '4f6efaef-284a-4680-9f35-8b65921ac0e1',
 '64d93164-d259-42a7-a21e-9c06a831c1d2',
 '2a105473-2edb-4464-baba-b10f72f425bd',
 '0b5ddc9c-712c-4724-a103-6178a2012163',
 '89bdcccd-5a3a-49d4-9bb2-900bf620f1d6',
 'f5403a12-5d67-41f9-80e1-d69c9d4c6904',
 '18a2fb43-1a6c-493f-acfc-955fac5bd8eb',
 '94a400da-634c-41f4-a689-caf03d29a17c',
 'f63761ea-51db-4a3e-8945-f0eeaba89cf7',
 '4d0c6f36-99aa-4223-97b0-6add22d0030f',
 '5f18b3e0-809a-44b3-aad2-4ca0add5f8aa',
 'a38a4db2-0a2a-4f9f-90ac-f5f42000580e',
 '849fa1f5-f9c2-4561-ac04-c2416782027d',
 '1024117a-0862-425d-ad93-053b45f43c67',
 '3b45ee29-d2d6-4b9c-aed5-cbbdfbf31115',
 '9c56dc21-e50a-49f0-8f75-788c793e1a64',
 '3093e1eb-e38f-